In [1]:
from py2neo import Graph, Node, Relationship
import pandas as pd

In [2]:
comments_within_2_minutes = pd.read_csv("comments_within_2_minutes.csv")

In [3]:
comments_within_2_minutes.shape

(11390, 27)

In [9]:
comments_within_2_minutes.columns

Index(['channel_id', 'channel_title', 'playlist_id', 'video_id', 'category_id',
       'video_published_at', 'title', 'description', 'thumbnail', 'view_count',
       'like_count_x', 'favorite_count', 'comment_count', 'tags', 'comment_id',
       'parent_id', 'comment_text', 'author_name', 'author_channel_id',
       'author_url', 'comment_published_at', 'can_rate', 'can_reply',
       'like_count_y', 'reply_count', 'is_public', 'time_difference'],
      dtype='object')

In [12]:
# drop null and nan values in comment_id column
comments_within_2_minutes = comments_within_2_minutes[comments_within_2_minutes['comment_id'].notna()]

In [14]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

In [15]:
for index, row in comments_within_2_minutes.iterrows():
    
    # channel node and relationship
    channel_node = Node(
        "Channel", channel_id=row["channel_id"], channel_title=row["channel_title"]
    )
    graph.merge(channel_node, "Channel", "channel_id")

    # video node and relationship
    video_node = Node(
        "Video",
        video_id=row["video_id"],
        view_count=row["view_count"],
        like_count=row["like_count_x"],
    )
    graph.merge(video_node, "Video", "video_id")
    relationship_channel_video = Relationship(channel_node, "POSTED", video_node)
    graph.merge(relationship_channel_video, "Channel", "channel_id")

    # comment node and relationship
    comment_node = Node(
        "Comment", comment_id=row["comment_id"], comment_text=row["comment_text"]
    )
    graph.merge(comment_node, "Comment", "comment_id")
    relationship_video_comment = Relationship(video_node, "HAS_COMMENT", comment_node)
    graph.merge(relationship_video_comment, "Video", "video_id")